# Part 1 - Deal with the Wikipedia Table

Import modules first.

In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Request data from the Wikipeida page. Use BeautifulSoup to get the html content, and apply __soup.find__ to get the part corresponding to the target table.

In [75]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
soup = BeautifulSoup(req.text, "html5lib")
data = soup.find(class_="wikitable sortable")

Define a new dataframe __neighborhoods__ with the desired columns.

In [76]:
column_names = ['Postal Code', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

Extract data of the table row by row. For each row, I record data into dataframe only if the corresponding borough is assigned.

When the loop reaches the end of the table, __the postal code on next line__ is no longer a string of 3 characters, indicating the end of data recording.

In [77]:
header = data.findChild("tr")
dataline = header.find_next('tr')

while True:
    post0 = dataline.find_next('td')
    post = post0.get_text(strip=True) 
    
    bor0 = post0.find_next('td')
    bor = bor0.get_text(strip=True)
    # skip the data whose borough is not assigned     
    if bor != 'Not assigned':
        neigh0 = bor0.find_next('td')
        neigh = neigh0.get_text(strip=True)
            
        neighborhoods = neighborhoods.append({'Postal Code': post,
                                              'Borough': bor,
                                              'Neighborhood': neigh}, ignore_index=True)
    dataline = dataline.find_next('tr') 
    post_check = dataline.find_next('td').get_text(strip=True)
    # reach the end of the table
    if len(post_check) != 3:
        break

The first ten rows of the dataframe __neighborhoods__:

In [78]:
neighborhoods.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


The row count of the dataframe __neighborhoods__:

In [79]:
print('The number of rows in the dataframe: ',neighborhoods.shape[0])

The number of rows in the dataframe:  103


# Part 2 - Add Latitude and Longitude 